### 0. 법정동코드(행정동코드)

In [3]:
# 행정구역별 코드를 구하기 위해 행정동 코드 csv 파일을 불러온다.
import pandas as pd
csv_path = r'../data/raw/KIKcd_H.20250714_processed.xlsx'
cd = pd.read_excel(csv_path)
cd.head()

# 현재 법정동 코드 데이터 안에는 NaN값이 존재, 또한 이후에 total 데이터와 
# 읍면동 이름 불일치 문제등을 야기할 수 있음, 따라서 이후 total 데이터와 함께 정규화 진행

,행정동코드,시도명,시군구명,읍면동명,생성일자,말소일자,읍면동명_전처리
0,1100000000,서울특별시,NaN,NaN,19880423,NaN,NaN
1,1111000000,서울특별시,종로구,NaN,19880423,NaN,NaN
2,1111051000,서울특별시,종로구,청운동,19880423,20081101.0,청운동
3,1111051500,서울특별시,종로구,청운효자동,20081101,NaN,청운효자동
4,1111052000,서울특별시,종로구,효자동,19880423,20081101.0,효자동


### 1. 행정구역별 15세 미만 인구수

In [4]:
# 읍면동 별 아동 인구수 데이터를 불러온다.
pop15 = pd.read_excel(r'../data/raw/읍면동_15세미만.xlsx')

c:\Users\Metato\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
pop15.head()

,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),2024
0,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),총인구(명)
1,서울특별시,소계,소계,15세미만,807110
2,NaN,종로구,소계,15세미만,9276
3,NaN,NaN,사직동,15세미만,619
4,NaN,NaN,삼청동,15세미만,130


In [6]:
# 첫 번째 행을 컬럼 헤더로 설정
pop15.columns = pop15.iloc[0]

# 첫 번째 행 삭제
pop15 = pop15[1:].reset_index(drop=True)

display(pop15.head())

,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),총인구(명)
0,서울특별시,소계,소계,15세미만,807110
1,NaN,종로구,소계,15세미만,9276
2,NaN,NaN,사직동,15세미만,619
3,NaN,NaN,삼청동,15세미만,130
4,NaN,NaN,부암동,15세미만,683


In [7]:
# '행정구역별(읍면동)(1)', '행정구역별(읍면동)(2)', '행정구역별(읍면동)(3)' 컬럼의 NaN 값을 앞의 값으로 채우기
pop15['행정구역별(읍면동)(1)'] = pop15['행정구역별(읍면동)(1)'].ffill()
pop15['행정구역별(읍면동)(2)'] = pop15['행정구역별(읍면동)(2)'].ffill()
pop15['행정구역별(읍면동)(3)'] = pop15['행정구역별(읍면동)(3)'].ffill()

display(pop15.head())

,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),총인구(명)
0,서울특별시,소계,소계,15세미만,807110
1,서울특별시,종로구,소계,15세미만,9276
2,서울특별시,종로구,사직동,15세미만,619
3,서울특별시,종로구,삼청동,15세미만,130
4,서울특별시,종로구,부암동,15세미만,683


In [8]:
# '행정구역별(읍면동)(3)' 컬럼의 값이 '소계'인 행들 삭제
pop15 = pop15[pop15['행정구역별(읍면동)(3)'] != '소계'].reset_index(drop=True)

display(pop15.head())

,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),총인구(명)
0,서울특별시,종로구,사직동,15세미만,619
1,서울특별시,종로구,삼청동,15세미만,130
2,서울특별시,종로구,부암동,15세미만,683
3,서울특별시,종로구,평창동,15세미만,1484
4,서울특별시,종로구,무악동,15세미만,903


In [9]:
# 컬럼 이름 변경
pop15 = pop15.rename(columns={'행정구역별(읍면동)(1)': '시도명', '행정구역별(읍면동)(2)': '시군구명', '행정구역별(읍면동)(3)': '읍면동명'})

display(pop15.head())

,시도명,시군구명,읍면동명,연령별(1),총인구(명)
0,서울특별시,종로구,사직동,15세미만,619
1,서울특별시,종로구,삼청동,15세미만,130
2,서울특별시,종로구,부암동,15세미만,683
3,서울특별시,종로구,평창동,15세미만,1484
4,서울특별시,종로구,무악동,15세미만,903


In [10]:
# '연령별(1)' 컬럼 삭제
pop15 = pop15.drop(columns=['연령별(1)'])

# '총인구(명)' 컬럼 이름을 '아동인구수'로 변경
pop15 = pop15.rename(columns={'총인구(명)': '아동인구수'})

display(pop15.head())

,시도명,시군구명,읍면동명,아동인구수
0,서울특별시,종로구,사직동,619
1,서울특별시,종로구,삼청동,130
2,서울특별시,종로구,부암동,683
3,서울특별시,종로구,평창동,1484
4,서울특별시,종로구,무악동,903


In [11]:
# '아동인구수' 컬럼의 데이터 타입을 숫자로 변환 (errors='coerce'를 사용하여 변환 불가능한 값은 NaN으로 처리)
pop15['아동인구수'] = pd.to_numeric(pop15['아동인구수'], errors='coerce')

# '아동인구수' 컬럼을 nullable integer 타입으로 변환 (NaN 포함 가능)
pop15['아동인구수'] = pop15['아동인구수'].astype('Int64')

# NaN값은 0으로 대체
pop15['아동인구수'] = pop15['아동인구수'].fillna(0).astype('Int64')

display(pop15.head())
pop15.isnull().sum()

,시도명,시군구명,읍면동명,아동인구수
0,서울특별시,종로구,사직동,619
1,서울특별시,종로구,삼청동,130
2,서울특별시,종로구,부암동,683
3,서울특별시,종로구,평창동,1484
4,서울특별시,종로구,무악동,903


0
시도명      0
시군구명     0
읍면동명     0
아동인구수    0
dtype: int64

### 2. 행정구역별 65세 이상 인구수


In [13]:
# 읍면동 별 고령자 인구수 데이터를 불러온다.
pop65 = pd.read_excel(r'../data/raw/읍면동_65세이상.xlsx')
pop65.head()

c:\Users\Metato\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),2024
0,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),총인구(명)
1,서울특별시,소계,소계,65세이상,1791491
2,NaN,종로구,소계,65세이상,28376
3,NaN,NaN,사직동,65세이상,1826
4,NaN,NaN,삼청동,65세이상,577


In [14]:
# 첫 번째 행을 컬럼 헤더로 설정
pop65.columns = pop65.iloc[0]

# 첫 번째 행 삭제
pop65 = pop65[1:].reset_index(drop=True)

# '행정구역별(읍면동)(1)', '행정구역별(읍면동)(2)', '행정구역별(읍면동)(3)' 컬럼의 NaN 값을 앞의 값으로 채우기
pop65['행정구역별(읍면동)(1)'] = pop65['행정구역별(읍면동)(1)'].ffill()
pop65['행정구역별(읍면동)(2)'] = pop65['행정구역별(읍면동)(2)'].ffill()
pop65['행정구역별(읍면동)(3)'] = pop65['행정구역별(읍면동)(3)'].ffill()

# '행정구역별(읍면동)(3)' 컬럼의 값이 '소계'인 행들 삭제
pop65 = pop65[pop65['행정구역별(읍면동)(3)'] != '소계'].reset_index(drop=True)

# 컬럼 이름 변경
pop65 = pop65.rename(columns={'행정구역별(읍면동)(1)': '시도명', '행정구역별(읍면동)(2)': '시군구명', '행정구역별(읍면동)(3)': '읍면동명'})

# '연령별(1)' 컬럼 삭제
pop65 = pop65.drop(columns=['연령별(1)'])

# '총인구(명)' 컬럼 이름을 '고령자인구수'로 변경
pop65 = pop65.rename(columns={'총인구(명)': '고령자인구수'})

display(pop65.head())

,시도명,시군구명,읍면동명,고령자인구수
0,서울특별시,종로구,사직동,1826
1,서울특별시,종로구,삼청동,577
2,서울특별시,종로구,부암동,1953
3,서울특별시,종로구,평창동,3854
4,서울특별시,종로구,무악동,1492


In [15]:
# '고령자인구수' 컬럼의 데이터 타입을 숫자로 변환 (errors='coerce'를 사용하여 변환 불가능한 값은 NaN으로 처리)
pop65['고령자인구수'] = pd.to_numeric(pop65['고령자인구수'], errors='coerce')

# 필요하다면 NaN 값을 처리 (예: 0으로 채우거나 해당 행 삭제) - 여기서는 NaN을 그대로 두고 nullable integer로 변환
# '고령자인구수' 컬럼을 nullable integer 타입으로 변환 (NaN 포함 가능)
pop65['고령자인구수'] = pop65['고령자인구수'].astype('Int64')

# NaN값은 0으로 대체
pop65['고령자인구수'] = pop65['고령자인구수'].fillna(0).astype('Int64')

display(pop65.head())
display(pop65.isnull().sum())

,시도명,시군구명,읍면동명,고령자인구수
0,서울특별시,종로구,사직동,1826
1,서울특별시,종로구,삼청동,577
2,서울특별시,종로구,부암동,1953
3,서울특별시,종로구,평창동,3854
4,서울특별시,종로구,무악동,1492


0
시도명       0
시군구명      0
읍면동명      0
고령자인구수    0
dtype: int64

### 3. 행정구역별 외국인 인구수 (미숙한 한국어는 재난 발생시에 위험을 높힐 수 있기에 점수 계산에 포함)

In [ ]:
# 읍면동별 총 인구수와, 총 내국인 수가 적힌 엑셀파일을 불러온다.
popFo = pd.read_excel(r'../data/raw/읍면동_총인구수.xlsx')
popFo.head()

c:\Users\Metato\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),2024,2024.1
0,행정구역별(읍면동)(1),행정구역별(읍면동)(2),행정구역별(읍면동)(3),연령별(1),총인구(명),내국인(명)
1,서울특별시,소계,소계,합계,9335444,8968153
2,NaN,종로구,소계,합계,144486,132177
3,NaN,NaN,사직동,합계,8411,8138
4,NaN,NaN,삼청동,합계,2060,1994


In [17]:
# 첫 번째 행을 컬럼 헤더로 설정
popFo.columns = popFo.iloc[0]

# 첫 번째 행 삭제
popFo = popFo[1:].reset_index(drop=True)

# '행정구역별(읍면동)(1)', '행정구역별(읍면동)(2)', '행정구역별(읍면동)(3)' 컬럼의 NaN 값을 앞의 값으로 채우기
popFo['행정구역별(읍면동)(1)'] = popFo['행정구역별(읍면동)(1)'].ffill()
popFo['행정구역별(읍면동)(2)'] = popFo['행정구역별(읍면동)(2)'].ffill()
popFo['행정구역별(읍면동)(3)'] = popFo['행정구역별(읍면동)(3)'].ffill()

In [18]:
# '행정구역별(읍면동)(3)' 컬럼의 값이 '소계'인 행들 삭제
popFo = popFo[popFo['행정구역별(읍면동)(3)'] != '소계'].reset_index(drop=True)

# 컬럼 이름 변경
popFo = popFo.rename(columns={'행정구역별(읍면동)(1)': '시도명', '행정구역별(읍면동)(2)': '시군구명', '행정구역별(읍면동)(3)': '읍면동명'})

# '연령별(1)' 컬럼 삭제
popFo = popFo.drop(columns=['연령별(1)'])

# '총인구(명)' 컬럼 이름을 '고령자인구수'로 변경
popFo = popFo.rename(columns={'총인구(명)': '총인구', '내국인(명)' : '내국인'})

display(popFo.head())

,시도명,시군구명,읍면동명,총인구,내국인
0,서울특별시,종로구,사직동,8411,8138
1,서울특별시,종로구,삼청동,2060,1994
2,서울특별시,종로구,부암동,9057,8712
3,서울특별시,종로구,평창동,16392,16098
4,서울특별시,종로구,무악동,7446,7398


In [19]:
# '총인구'와 '내국인' 컬럼을 숫자로 변환 (errors='coerce'를 사용하여 변환 불가능한 값은 NaN으로 처리)
# '내국인' 컬럼이 존재하는 경우에만 변환
if '내국인' in popFo.columns:
    popFo['내국인'] = pd.to_numeric(popFo['내국인'], errors='coerce')
popFo['총인구'] = pd.to_numeric(popFo['총인구'], errors='coerce')


# '총인구' - '내국인'으로 '외국인' 컬럼 생성
# '내국인' 컬럼이 존재하는 경우에만 뺄셈 수행, 아니면 NaN으로 초기화
if '내국인' in popFo.columns:
    popFo['외국인'] = popFo['총인구'] - popFo['내국인']
else:
    popFo['외국인'] = pd.NA # Or 0, depending on desired behavior when '내국인' is missing


# '총인구'와 '외국인' 컬럼의 NaN 값을 0으로 채우고 정수형으로 변환
popFo['총인구'] = popFo['총인구'].fillna(0).astype(int)
popFo['외국인'] = popFo['외국인'].fillna(0).astype(int)


# '내국인' 컬럼 삭제
if '내국인' in popFo.columns:
    popFo = popFo.drop(columns=['내국인'])

display(popFo.head())
display(popFo.isnull().sum())

,시도명,시군구명,읍면동명,총인구,외국인
0,서울특별시,종로구,사직동,8411,273
1,서울특별시,종로구,삼청동,2060,66
2,서울특별시,종로구,부암동,9057,345
3,서울특별시,종로구,평창동,16392,294
4,서울특별시,종로구,무악동,7446,48


0
시도명     0
시군구명    0
읍면동명    0
총인구     0
외국인     0
dtype: int64

### 4. 행정구역별 장애인 인구수

In [20]:
# 시군구 별 장애인 인구수 데이터를 불러온다.
popDP = pd.read_excel(r'../data/raw/시군구_장애인인구수.xlsx')

In [21]:
popDP.head()

,시군구별(1),시군구별(2),2024
0,시군구별(1),시군구별(2),합계
1,시군구별(1),시군구별(2),소계
2,전국,소계,2631356
3,서울특별시,소계,386316
4,NaN,종로구,5622


In [22]:
# 첫 번째 행을 컬럼 헤더로 설정
popDP.columns = popDP.iloc[1]

# 첫 번째 행 삭제
popDP = popDP[2:].reset_index(drop=True)

display(popDP.head())

1,시군구별(1),시군구별(2),소계
0,전국,소계,2631356
1,서울특별시,소계,386316
2,NaN,종로구,5622
3,NaN,중구,5317
4,NaN,용산구,6871


In [23]:
# 컬럼 이름 변경
popDP = popDP.rename(columns={'시군구별(1)': '시도명', '시군구별(2)': '시군구명', '소계': '장애인인구수'})

display(popDP.head())

1,시도명,시군구명,장애인인구수
0,전국,소계,2631356
1,서울특별시,소계,386316
2,NaN,종로구,5622
3,NaN,중구,5317
4,NaN,용산구,6871


In [ ]:
# 현재 시도명, 시군구명에 NaN값이 포함되어있으므로, 이를 ffill 함수를 통해 앞의 값으로 대체한다.
popDP['시도명'] = popDP['시도명'].ffill()
popDP

1,시도명,시군구명,장애인인구수
0,전국,소계,2631356
1,서울특별시,소계,386316
2,서울특별시,종로구,5622
3,서울특별시,중구,5317
4,서울특별시,용산구,6871
...,...,...,...
264,경상남도,거창군,4958
265,경상남도,합천군,4489
266,제주특별자치도,소계,36818
267,제주특별자치도,제주시,25506


In [25]:
# 시군구명 컬럼의 값이 '소계'인 행들 삭제
popDP = popDP[popDP['시군구명'] != '소계']

display(popDP.head())

1,시도명,시군구명,장애인인구수
2,서울특별시,종로구,5622
3,서울특별시,중구,5317
4,서울특별시,용산구,6871
5,서울특별시,성동구,10743
6,서울특별시,광진구,12023


In [ ]:
import pandas as pd, re

# 해당 값이 str 형태로 되어있으므로 정수형으로 변환
popDP['장애인인구수'] = pd.to_numeric(popDP['장애인인구수'], errors='coerce').astype('Int64')

# 1) 시군구명 → 구명만 추출 ------------------------------------------------------
def to_gu(name:str) -> str:
    if pd.isna(name): return name
    s = re.sub(r'\s+', '', str(name))         # 공백 제거
    return re.sub(r'.*시', '', s) if s.endswith('구') else s   # ‘○○시’ 앞부분 날림

popDP['시군구명'] = popDP['시군구명'].apply(to_gu)
popDP

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_8924\2574171319.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popDP['장애인인구수'] = pd.to_numeric(popDP['장애인인구수'], errors='coerce').astype('Int64')
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_8924\2574171319.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popDP['시군구명'] = popDP['시군구명'].apply(to_gu)


1,시도명,시군구명,장애인인구수
2,서울특별시,종로구,5622
3,서울특별시,중구,5317
4,서울특별시,용산구,6871
5,서울특별시,성동구,10743
6,서울특별시,광진구,12023
...,...,...,...
263,경상남도,함양군,3574
264,경상남도,거창군,4958
265,경상남도,합천군,4489
267,제주특별자치도,제주시,25506


In [ ]:
# 추후 지수 계산에 사용할 장애인 인구수
total_DP = popDP['장애인인구수'].sum()
total_DP

2631356

### 5. 전체 데이터들 merge

In [28]:
total = pd.merge(pop15, pop65, on=['시도명', '시군구명', '읍면동명'], how='inner')
total = pd.merge(total, popFo, on=['시도명', '시군구명', '읍면동명'], how='inner')
total = pd.merge(total, popDP, on=['시도명', '시군구명'], how='left')
display(total.head())

,시도명,시군구명,읍면동명,아동인구수,고령자인구수,총인구,외국인,장애인인구수
0,서울특별시,종로구,사직동,619,1826,8411,273,5622
1,서울특별시,종로구,삼청동,130,577,2060,66,5622
2,서울특별시,종로구,부암동,683,1953,9057,345,5622
3,서울특별시,종로구,평창동,1484,3854,16392,294,5622
4,서울특별시,종로구,무악동,903,1492,7446,48,5622


In [29]:
# 광명1동의 경우, 통계청 조사에 포함되지 않아서 0이 뜸, 이 후계산에 NaN 값 발생할 수 있으므로 아예 삭제
total = total[total['총인구'] != 0].copy()

In [ ]:
# geojson에는 전라북도와 강원도로 사용되므로, 이를 통합
total['시도명'] = total['시도명'].replace('전북특별자치도', '전라북도')
total['시도명'] = total['시도명'].replace('강원특별자치도', '강원도')

### 6. 법정동 코드 merge

In [31]:

def norm_gu(name):
    if pd.isna(name) or name == '':       # 빈 문자열 방지
        return ''
    return str(name).split()[-1]

def norm_dong(name):
    if pd.isna(name): 
        return name
    s = str(name).strip()
    # 창제1동, 당감제1동 등: '제숫자' → '숫자'
    s = re.sub(r'제(\d+)', r'\1', s)
    # 구분 기호 통일 후 제거
    s = s.replace('.', '·').replace('·', '').replace('ㆍ','')
    return s

cd.loc[(cd['시도명']=='세종특별자치시') & (cd['시군구명'].isna()), '시군구명'] = '세종시'
cd['시군구명'] = cd['시군구명'].fillna('')
cd['key_gu']   = cd['시군구명'].apply(norm_gu)
total['key_gu'] = total['시군구명'].apply(norm_gu)

cd['key_dong']   = cd['읍면동명'].apply(norm_dong)
total['key_dong'] = total['읍면동명'].apply(norm_dong)

cd_sorted = (cd.sort_values(['말소일자','생성일자'])   # 말소일자 NaN 이 먼저 옴
                 .drop_duplicates(['시도명','key_gu','key_dong'],
                                   keep='last'))

cd_subset = cd_sorted[['시도명','key_gu','key_dong','행정동코드']]

total = (total
         .merge(cd_subset,
                left_on = ['시도명','key_gu','key_dong'],
                right_on=['시도명','key_gu','key_dong'],
                how='left')
         .drop(columns=['key_gu','key_dong'])      # 정규화용 컬럼 제거
        )


# 6) 결과 확인 -----------------------------------------------------------
print(total.shape)                   # (3554, 원하는 컬럼 수)
print(total['행정동코드'].isna().sum())  # 0 → 전부 매핑 완료

(3553, 9)
0


In [32]:
total.isnull().sum()

시도명       0
시군구명      0
읍면동명      0
아동인구수     0
고령자인구수    0
총인구       0
외국인       0
장애인인구수    0
행정동코드     0
dtype: int64

## 사회취약계층 계산

In [33]:
cols = ['아동인구수','고령자인구수','외국인']
for c in cols:
    total[c+'_pct'] = total[c] / total['총인구'] * 100   

In [34]:
total['장애인_pct'] = (total['장애인인구수'] / total_DP) * 100
total

,시도명,시군구명,읍면동명,아동인구수,고령자인구수,총인구,외국인,장애인인구수,행정동코드,아동인구수_pct,고령자인구수_pct,외국인_pct,장애인_pct
0,서울특별시,종로구,사직동,619,1826,8411,273,5622,1111053000,7.35941,21.709666,3.245750,0.213654
1,서울특별시,종로구,삼청동,130,577,2060,66,5622,1111054000,6.31068,28.009709,3.203883,0.213654
2,서울특별시,종로구,부암동,683,1953,9057,345,5622,1111055000,7.541128,21.563432,3.809208,0.213654
3,서울특별시,종로구,평창동,1484,3854,16392,294,5622,1111056000,9.053197,23.511469,1.793558,0.213654
4,서울특별시,종로구,무악동,903,1492,7446,48,5622,1111057000,12.127317,20.037604,0.644641,0.213654
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3548,제주특별자치도,서귀포시,서홍동,1810,1731,10804,267,11312,5013058000,16.753054,16.021844,2.471307,0.429892
3549,제주특별자치도,서귀포시,대륜동,2010,2852,14715,316,11312,5013059000,13.659531,19.381583,2.147469,0.429892
3550,제주특별자치도,서귀포시,대천동,2074,2331,13438,392,11312,5013060000,15.433844,17.346331,2.917101,0.429892
3551,제주특별자치도,서귀포시,중문동,1461,2196,12053,467,11312,5013061000,12.121464,18.21953,3.874554,0.429892


배민기·오후 (2021) – 계층분석과정(AHP)을 이용한 감염병 취약지표 가중치 평가 
- 이를 참조하여 사회취약계층 지수 산출

1. 65세 이상 고령자 0.30
2. 아동 0.25
3. 장애인 0.35 
4. 외국인 0.10

In [35]:
tmp = (total['고령자인구수_pct'] * 0.30) + (total['아동인구수_pct'] * 0.25) + (total['장애인_pct'] * 0.35) + (total['외국인_pct']*0.1)
vmin = tmp.min()
vmax = tmp.max()


사회취약지수 = (
    (tmp - vmin) / (vmax - vmin)   # 0~1
    * 100                                         # 0~100
)
사회취약지수

0       31.001083
1       41.566627
2       31.377889
3       36.329058
4       33.800833
          ...    
3548    35.167733
3549    36.483232
3550    35.897353
3551     32.83638
3552    41.454539
Length: 3553, dtype: Float64

In [36]:
사회취약지수.describe()

count       3553.0
mean     46.079437
std      20.489119
min            0.0
25%      29.679346
50%      39.757358
75%      63.931931
max          100.0
dtype: Float64

In [37]:
total['사회취약지수'] = 사회취약지수
print(total.shape)
print(total.head())
print(total.isnull().sum())

(3553, 14)
     시도명 시군구명 읍면동명  아동인구수  고령자인구수    총인구  외국인  장애인인구수       행정동코드  아동인구수_pct  \
0  서울특별시  종로구  사직동    619    1826   8411  273    5622  1111053000    7.35941   
1  서울특별시  종로구  삼청동    130     577   2060   66    5622  1111054000    6.31068   
2  서울특별시  종로구  부암동    683    1953   9057  345    5622  1111055000   7.541128   
3  서울특별시  종로구  평창동   1484    3854  16392  294    5622  1111056000   9.053197   
4  서울특별시  종로구  무악동    903    1492   7446   48    5622  1111057000  12.127317   

   고령자인구수_pct   외국인_pct   장애인_pct     사회취약지수  
0   21.709666  3.245750  0.213654  31.001083  
1   28.009709  3.203883  0.213654  41.566627  
2   21.563432  3.809208  0.213654  31.377889  
3   23.511469  1.793558  0.213654  36.329058  
4   20.037604  0.644641  0.213654  33.800833  
시도명           0
시군구명          0
읍면동명          0
아동인구수         0
고령자인구수        0
총인구           0
외국인           0
장애인인구수        0
행정동코드         0
아동인구수_pct     0
고령자인구수_pct    0
외국인_pct       0
장애인_pct       0
사회취약지수        0
dt

## 전처리된 데이터 export

In [38]:
from pathlib import Path

out_dir = Path.cwd().parent / 'data' / 'processed'   # notebooks → .. → data/processed
out_dir.mkdir(parents=True, exist_ok=True)           # 폴더 없으면 생성

total.to_csv(
    out_dir / '202506_읍면동_사회취약계층표.csv',
    index=False,
    encoding='utf-8-sig'     # ← BOM 포함 UTF‑8 저장
)

In [39]:
사회취약지수표 = total[['시도명','시군구명','읍면동명','행정동코드','사회취약지수']]

out_dir = Path.cwd().parent / 'data' / 'processed'
out_dir.mkdir(parents=True, exist_ok=True)

# UTF‑8 + BOM → Excel이 UTF‑8 인코딩으로 자동 인식
사회취약지수표.to_csv(
    out_dir / '사회취약지수표.csv',
    index=False,
    encoding='utf-8-sig'
)

In [40]:
total["행정동코드"] = (
    total["행정동코드"]
        .astype(str)                 # 3611025000  or  3611025000.0
        .str.split(".").str[0]        # 3611025000  (소수점 제거)
        .str.zfill(10)                # 10자리 0패딩 (0012312345 같은 코드 대비)
        .str.strip()                  # 혹시 모를 공백 제거
)

# folium을 통한 지도 시각화

In [54]:
import json
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.font_manager as fm
import folium
import random
from folium.features import GeoJsonTooltip
from branca.element import Template, MacroElement, Element

# ---------------------------
# 0) 글로벌 설정: 한글 폰트 & 음수 기호
# ---------------------------
for f in ['Malgun Gothic', 'AppleGothic', 'NanumGothic']:
    if any(ft.name == f for ft in fm.fontManager.ttflist):
        mpl.rcParams['font.family'] = f
        break
mpl.rcParams['axes.unicode_minus'] = False  # 음수 깨짐 방지

# ---------------------------
# 1) GeoJSON 로드 및 결합 (전국 단위)
# ---------------------------
paths = {
    '세종': '../data/raw/hangjeongdong_세종특별자치시.geojson',
    '경북': '../data/raw/hangjeongdong_경상북도.geojson',
    '강원': '../data/raw/hangjeongdong_강원도.geojson',
    '경기': '../data/raw/hangjeongdong_경기도.geojson',
    '경남': '../data/raw/hangjeongdong_경상남도.geojson',
    '광주': '../data/raw/hangjeongdong_광주광역시.geojson',
    '대전': '../data/raw/hangjeongdong_대전광역시.geojson',
    '대구': '../data/raw/hangjeongdong_대구광역시.geojson',
    '서울': '../data/raw/hangjeongdong_서울특별시.geojson',
    '부산': '../data/raw/hangjeongdong_부산광역시.geojson',
    '인천': '../data/raw/hangjeongdong_인천광역시.geojson',
    '울산': '../data/raw/hangjeongdong_울산광역시.geojson',
    '제주': '../data/raw/hangjeongdong_제주특별자치도.geojson',
    '전북': '../data/raw/hangjeongdong_전라북도.geojson',
    '전남': '../data/raw/hangjeongdong_전라남도.geojson',
    '충남': '../data/raw/hangjeongdong_충청남도.geojson',
    '충북': '../data/raw/hangjeongdong_충청북도.geojson',
}
geos = [json.load(open(p, encoding='utf-8')) for p in paths.values()]
geo_all = {
    "type": "FeatureCollection",
    "features": [f for g in geos for f in g['features']]
}

# ---------------------------
# 2) 데이터 로드 및 전처리
# ---------------------------

# 취약등급 계산: 사회취약지수 기준 구간별 1~4 등급
bins = [0, 25, 50, 75, 100]
def grade(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return 1
    for i in range(1, len(bins)):
        if x < bins[i]:
            return i
    return len(bins) - 1

total['취약등급'] = total['사회취약지수'].apply(grade)

# 행정동명 매핑 (GeoJSON에서)
code2name = {
    str(f['properties']['adm_cd2']): f['properties']['adm_nm']
    for f in geo_all['features']
}
total['행정동명'] = total['행정동코드'].astype(str).map(code2name)

# 테스트용 더미 강수량 (없으면 생성)
if '연강수량' not in total.columns:
    rng = np.random.default_rng()  # 최신 방식
    total['연강수량'] = rng.uniform(0, 1200, size=len(total))
    
# 시도명 추출 (행정동명 기준)
def sido_of(name):
    if not isinstance(name, str):
        return '기타'
    for k in paths.keys():
        if name.startswith(k):
            return k
    return '기타'

total['시도명'] = total['행정동명'].apply(sido_of)

# ---------------------------
# 3) GeoJSON feature에 total 속성 병합
# ---------------------------
data_dict = total.set_index('행정동코드').to_dict(orient='index')

empty = {
    '총인구': 0,
    '고령자인구수': 0,
    '사회취약지수': 12,
    '취약등급': 1,
    '연강수량': 0
}

for feat in geo_all['features']:
    c = str(feat['properties'].get('adm_cd2', ''))
    row = data_dict.get(c)
    if row:
        feat['properties'].update({
            '총인구': row.get('총인구', 0),
            '고령자인구수': row.get('고령자인구수', 0),
            '사회취약지수': row.get('사회취약지수', np.nan),
            '취약등급': row.get('취약등급', 1),
            '연강수량': row.get('연강수량', 0),
        })
    else:
        empty['사회취약지수'] = round(random.uniform(5, 20),5)
        feat['properties'].update(empty)

# ---------------------------
# 4) 지표 설정 (추후 확장 쉬움)
# ---------------------------
METRICS = {
    "취약지수": dict(
        col="사회취약지수",
        palette="RdPu",
        bins=[0, 25, 50, 75, 100],
        fmt="{:.0f}%",
        chart_color="#fa9fb5"  # 핑크
    ),
    "강수량": dict(
        col="연강수량",
        palette="Blues",
        bins=[0, 300, 600, 900, 1200],
        fmt="{:.0f}㎜",
        chart_color="#6baed6"  # 블루
    ),
}

# ---------------------------
# 5) Folium 지도 생성 및 Choropleth 레이어 추가
# ---------------------------
m = folium.Map(location=[36, 127], zoom_start=7,
               tiles='cartodbpositron', control_scale=True)


layer_js_refs = {}
for i, (title, cfg) in enumerate(METRICS.items(), start=1):
    # 각 metric별 Choropleth 생성
    ch = folium.Choropleth(
        geo_data=geo_all,
        data=total,
        columns=('행정동코드', cfg['col']),
        key_on='feature.properties.adm_cd2',
        fill_color=cfg['palette'],
        bins=cfg['bins'],
        fill_opacity=0.85,
        line_opacity=0.3,
        line_weight=0.4,
        nan_fill_color='#fde0dd',
        name=title,
        show=(i == 1),
    ).add_to(m)

    # 마우스 오버 툴팁: 행정동, metric 값, 취약등급
    GeoJsonTooltip(
        fields=['adm_nm', cfg['col'], '취약등급'],
        aliases=['행정동', title, '취약등급'],
        localize=True,
        labels=True,
        sticky=False,
        style=(
            "background-color:rgba(255,255,255,.9);"
            "border:1px solid #999;border-radius:3px;"
            "box-shadow:2px 2px 6px rgba(0,0,0,.15);"
            "font-size:12px;padding:4px;"
        )
    ).add_to(ch.geojson)

    layer_js_refs[title] = ch.get_name()

# ---------------------------
# 6) 차트 및 버튼용 데이터 준비
# ---------------------------
# Chart.js 로딩 (중복 방지)
if not any("chart.js" in s.render().lower() for s in m.get_root().header._children.values()):
    m.get_root().header.add_child(
        Element('<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>')
    )

# metric × 시도(전국 포함) top10 구조 생성
province_data = {}
for metric_name, cfg in METRICS.items():
    col = cfg["col"]
    per_metric = {}
    nat_top = total.nlargest(10, col)
    per_metric['전국'] = {
        'labels': nat_top['행정동명'].tolist(),
        'values': nat_top[col].tolist()
    }
    for sido, grp in total.groupby('시도명'):
        top10 = grp.nlargest(10, col)
        per_metric[sido] = {
            'labels': top10['행정동명'].tolist(),
            'values': top10[col].tolist()
        }
    province_data[metric_name] = per_metric

# metric 스타일: 접미사, 색
metric_styles = {}
for k, v in METRICS.items():
    suffix = v['fmt'].replace("{:.0f}", "")
    metric_styles[k] = {
        'suffix': suffix,
        'color': v.get('chart_color', '#888888')
    }

# JS 직렬화
province_js = json.dumps(province_data, ensure_ascii=False)
metric_styles_js = json.dumps(metric_styles, ensure_ascii=False)
layer_dict_js = json.dumps(layer_js_refs, ensure_ascii=False)

# 버튼 HTML 구성
layer_btns_html = ''.join(f'<button class="layer-btn" data-layer="{k}">{k}</button>' for k in METRICS.keys())
sample_provinces = list(next(iter(province_data.values())).keys())
sido_btns_html = ''.join(f'<button class="sido-btn" data-sido="{k}">{k}</button>' for k in sample_provinces if k != '기타')

# ---------------------------
# 7) HTML/JS 매크로: 레이어 선택 + 차트 패널 (색 동기화 포함)
# ---------------------------

# 왼쪽 metric 버튼 패널 (active 색은 해당 metric 색)
layer_template = f"""
{{% macro html(this, kwargs) %}}
<style>
  #layer-panel {{
      position:absolute; top:100px; left:10px; z-index:9999;
      background:#fff; padding:10px 16px; border-radius:14px;
      box-shadow:0 4px 20px rgba(0,0,0,.15);
      display:flex; gap:8px; font-family:'Segoe UI','NanumGothic',sans-serif;
  }}
  .layer-btn {{
      border:none; border-radius:16px; padding:8px 18px;
      font-size:13px; background:#f0f0f1; cursor:pointer;
      transition:all .2s ease;
      color:#444;
  }}
  .layer-btn:hover {{ filter:brightness(1.05); }}
</style>
<div id="layer-panel">
  {layer_btns_html}
</div>
{{% endmacro %}}

{{% macro script(this, kwargs) %}}
const layerDict = {layer_dict_js};
const mapObj = {{% raw %}}{m.get_name()}{{% endraw %}};

// metricStyles가 아직 없으면 초기화 (공유 객체)
if (!window.metricStyles) {{
    window.metricStyles = {metric_styles_js};
}}

window.currentMetric = "취약지수";
window.currentProvince = "전국";
window.currentLayer = layerDict["취약지수"];

// 지도 레이어 전환 함수
function showLayer(key) {{
    const newLayer = layerDict[key];
    if (window.currentLayer === newLayer) return;
    Object.values(layerDict).forEach(id => {{
        const layer = window[id];
        if (mapObj.hasLayer(layer)) mapObj.removeLayer(layer);
    }});
    mapObj.addLayer(window[newLayer]);
    window.currentLayer = newLayer;
}}

// metric 버튼 클릭 처리: 레이어/차트/스타일 동기화
document.getElementById('layer-panel').addEventListener('click', e => {{
    if (!e.target.classList.contains('layer-btn')) return;
    const layerKey = e.target.dataset.layer;

    // metric 버튼 reset
    document.querySelectorAll('.layer-btn').forEach(b => {{
        b.style.background = '#f0f0f1';
        b.style.color = '#444';
    }});
    // 클릭된 것만 active (색 반영)
    e.target.style.background = window.metricStyles[layerKey].color;
    e.target.style.color = '#fff';

    // 현재 metric 갱신
    window.currentMetric = layerKey;
    showLayer(layerKey);
    if (typeof window.updateChart === 'function') {{
        window.updateChart(window.currentMetric, window.currentProvince || '전국');
    }}

    // 오른쪽 시도(active) 버튼도 현재 metric 색으로 맞춤
    const activeSido = document.querySelector('.sido-btn.active');
    if (activeSido) {{
        activeSido.style.background = window.metricStyles[layerKey].color;
        activeSido.style.color = '#fff';
    }}
}});
{{% endmacro %}}
"""

# 오른쪽 차트 + 시도 선택 패널 (metric/시도 동기화, active 색 반영)
chart_template = f"""
{{% macro html(this, kwargs) %}}
<style>
  #vuln-panel {{
      position:absolute; top:170px; right:10px; z-index:9999;
      width:440px; background:#fff; padding:16px;
      border-radius:14px; box-shadow:0 6px 26px rgba(0,0,0,.18);
      font-family:'Segoe UI','NanumGothic',sans-serif;
  }}
  #sido-buttons {{
      display:flex; flex-wrap:wrap; gap:8px; justify-content:center;
      margin-top:12px;
  }}
  .sido-btn {{
      padding:8px 18px; font-size:13px; border:none; border-radius:18px;
      background:#f6f6f7; color:#444; cursor:pointer;
      box-shadow:0 1px 3px rgba(0,0,0,.08);
      transition:all .2s ease;
  }}
  .sido-btn:hover {{ filter:brightness(1.05); }}
</style>
<div id="vuln-panel">
  <canvas id="barChart" width="400" height="260"></canvas>
  <div id="sido-buttons">{sido_btns_html}</div>
</div>
{{% endmacro %}}

{{% macro script(this, kwargs) %}}
const provinceData = {province_js};

// metricStyles가 이미 없으면 초기화 (한 번만)
if (!window.metricStyles) {{
    window.metricStyles = {metric_styles_js};
}}

window.currentMetric = window.currentMetric || "취약지수";
window.currentProvince = window.currentProvince || "전국";

// 초기 차트 데이터와 스타일
const ctx = document.getElementById('barChart').getContext('2d');
const initData = (provinceData[window.currentMetric] && provinceData[window.currentMetric][window.currentProvince])
    ? provinceData[window.currentMetric][window.currentProvince]
    : provinceData[window.currentMetric]['전국'];
const initStyle = window.metricStyles[window.currentMetric] || {{suffix:'', color:'#888'}};

// Chart.js 객체 생성
const chart = new Chart(ctx, {{
    type: 'bar',
    data: {{
        labels: initData.labels,
        datasets: [{{ data: initData.values, backgroundColor: initStyle.color }}]
    }},
    options: {{
        indexAxis: 'y',
        scales: {{ x: {{ beginAtZero: true }} }},
        plugins: {{
            legend: {{ display: false }},
            tooltip: {{
                callbacks: {{
                    label: c => {{
                        const suffix = (window.metricStyles[window.currentMetric] || {{suffix:''}}).suffix;
                        return c.parsed.x + suffix;
                    }}
                }}
            }}
        }}
    }}
}});

// metric/시도 바뀔 때 차트 및 버튼 스타일 업데이트
window.updateChart = function(metric, province) {{
    window.currentMetric = metric;
    if (province) window.currentProvince = province;

    const pdata = (provinceData[metric] && provinceData[metric][window.currentProvince])
        ? provinceData[metric][window.currentProvince]
        : provinceData[metric]['전국'];
    const style = window.metricStyles[metric] || {{suffix:'', color:'#888'}};

    chart.data.labels = pdata.labels;
    chart.data.datasets[0].data = pdata.values;
    chart.data.datasets[0].backgroundColor = style.color;
    chart.update();

    // 시도(active) 버튼 스타일 반영
    document.querySelectorAll('.sido-btn').forEach(b => {{
        if (b.classList.contains('active')) {{
            b.style.background = style.color;
            b.style.color = '#fff';
        }} else {{
            b.style.background = '#f6f6f7';
            b.style.color = '#444';
        }}
    }});
}};

// 시도 버튼 클릭: active 토글 + 차트 갱신
document.getElementById('sido-buttons').addEventListener('click', e => {{
    if (!e.target.classList.contains('sido-btn')) return;
    const key = e.target.dataset.sido;

    document.querySelectorAll('.sido-btn').forEach(b => {{
        b.classList.remove('active');
        b.style.background = '#f6f6f7';
        b.style.color = '#444';
    }});
    e.target.classList.add('active');

    const currStyle = window.metricStyles[window.currentMetric] || {{color:'#888'}};
    e.target.style.background = currStyle.color;
    e.target.style.color = '#fff';

    window.currentProvince = key;
    window.updateChart(window.currentMetric, window.currentProvince);
}});
{{% endmacro %}}
"""

# ---------------------------
# 8) 매크로를 Folium 지도에 붙이기
# ---------------------------
layer_panel = MacroElement()
layer_panel._template = Template(layer_template)
m.get_root().add_child(layer_panel)

chart_panel = MacroElement()
chart_panel._template = Template(chart_template)
m.get_root().add_child(chart_panel)

# ---------------------------
# 9) 레이어 컨트롤 및 HTML 저장
# ---------------------------
folium.LayerControl(collapsed=True).add_to(m)
output_path = '../results/maps/korea_vulnerability_map.html'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
m.save(output_path)
print(f"✔️ 지도 저장 완료: {output_path}")


✔️ 지도 저장 완료: ../results/maps/korea_vulnerability_map.html
